## Adaptacion del dataset para el modelo

In [1]:
import pandas as pd
import json

## 1. Cargamos el archivo json

In [2]:
customer_file = '../data/customers.json'
df_json_nested = pd.read_json(customer_file, lines= True)
df_json_nested.head()


,fraudulent,customer,orders,paymentMethods,transactions
0,False,"{'customerEmail': 'josephhoward@yahoo.com', 'c...","[{'orderId': 'vjbdvd', 'orderAmount': 18, 'ord...","[{'paymentMethodId': 'wt07xm68b', 'paymentMeth...","[{'transactionId': 'a9lcj51r', 'orderId': 'vjb..."
1,True,"{'customerEmail': 'evansjeffery@yahoo.com', 'c...","[{'orderId': 'nlghpa', 'orderAmount': 45, 'ord...","[{'paymentMethodId': 'y3xp697jx', 'paymentMeth...","[{'transactionId': '5mi94sfw', 'orderId': 'nlg..."
2,False,"{'customerEmail': 'andersonwilliam@yahoo.com',...","[{'orderId': 'yk34y2', 'orderAmount': 33, 'ord...","[{'paymentMethodId': '8pneoi03z', 'paymentMeth...","[{'transactionId': 'q3lyvbza', 'orderId': 'yk3..."
3,False,"{'customerEmail': 'rubenjuarez@yahoo.com', 'cu...","[{'orderId': 'fbz9ep', 'orderAmount': 34, 'ord...","[{'paymentMethodId': 'pdxjdwui4', 'paymentMeth...","[{'transactionId': 'vx4cjc27', 'orderId': 'fbz..."
4,True,"{'customerEmail': 'uchen@malone.com', 'custome...","[{'orderId': '56h7iw', 'orderAmount': 71, 'ord...","[{'paymentMethodId': 'w1i1zq3rg', 'paymentMeth...","[{'transactionId': 'q8j3dgni', 'orderId': '56h..."


## 2. Customer dataframe

In [3]:
customers_df = pd.json_normalize(data = df_json_nested['customer'])
customers_df.head()

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress
0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896"
1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
2,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250"
3,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM..."
4,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,..."


## 3 Crear funcion generica para transformar listas anidadas en dataframe

In [4]:
def nested_to_df(df, column_name):
    nested = pd.DataFrame([md for md in df[column_name]])
    item_list=[]
    for index,row in nested.iterrows():
        for item in row:
            if item is not None:
                item_list.append(item)
    return pd.DataFrame(item_list)

## 4. Crear dataframe de orders

In [5]:
orders_df = nested_to_df(df_json_nested, 'orders')
orders_df.head()

,orderId,orderAmount,orderState,orderShippingAddress
0,vjbdvd,18,pending,"5493 Jones Islands\nBrownside, CA 51896"
1,yp6x27,26,fulfilled,"5493 Jones Islands\nBrownside, CA 51896"
2,nlghpa,45,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497"
3,uw0eeb,23,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
4,bn44oh,43,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801"


## 5. Crear dataframe de transactions

In [6]:
transactions_df = nested_to_df(df_json_nested, 'transactions')
transactions_df.head()

,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed
0,a9lcj51r,vjbdvd,wt07xm68b,18,False
1,y4wcv03i,yp6x27,wt07xm68b,26,False
2,5mi94sfw,nlghpa,41ug157xz,45,False
3,br8ba1nu,uw0eeb,41ug157xz,23,False
4,a33145ss,bn44oh,y3xp697jx,43,True


## 6. Crear dataframe payments

In [7]:
payments_df = nested_to_df(df_json_nested, 'paymentMethods')
payments_df.head()

,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer
0,wt07xm68b,True,card,JCB 16 digit,Citizens First Banks
1,y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks
2,6krszxc05,False,card,VISA 16 digit,Vertex Bancorp
3,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.
4,m52tx8e1s,False,card,Mastercard,Vertex Bancorp


## 7. Crear dataframe fraudulent

In [8]:
fraudulent_df = nested_to_df(df_json_nested, 'fraudulent')
fraudulent_df.columns = ['fraudulent']
fraudulent_df.head()

,fraudulent
0,False
1,True
2,False
3,False
4,True


## 8. Concatenar dataframes

In [9]:
data = pd.concat([customers_df, orders_df, payments_df, transactions_df, fraudulent_df], axis=1)
data.head()

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,orderId,orderAmount,orderState,orderShippingAddress,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,fraudulent
0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",vjbdvd,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",wt07xm68b,True,card,JCB 16 digit,Citizens First Banks,a9lcj51r,vjbdvd,wt07xm68b,18,False,False
1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,yp6x27,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks,y4wcv03i,yp6x27,wt07xm68b,26,False,True
2,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",nlghpa,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",6krszxc05,False,card,VISA 16 digit,Vertex Bancorp,5mi94sfw,nlghpa,41ug157xz,45,False,False
3,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",uw0eeb,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,br8ba1nu,uw0eeb,41ug157xz,23,False,False
4,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",bn44oh,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",m52tx8e1s,False,card,Mastercard,Vertex Bancorp,a33145ss,bn44oh,y3xp697jx,43,True,True


In [10]:
from funpymodeling import status
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,customerEmail,455,0.730337,0,0.000000,161,object
1,customerPhone,455,0.730337,0,0.000000,168,object
2,customerDevice,455,0.730337,0,0.000000,168,object
3,customerIPAddress,455,0.730337,0,0.000000,165,object
4,customerBillingAddress,455,0.730337,0,0.000000,166,object
5,orderId,145,0.232745,0,0.000000,478,object
6,orderAmount,145,0.232745,0,0.000000,67,float64
7,orderState,145,0.232745,0,0.000000,3,object
8,orderShippingAddress,145,0.232745,0,0.000000,274,object
9,paymentMethodId,281,0.451043,0,0.000000,342,object


## 9. Eliminar columnas duplicadas

In [11]:
data = data.loc[:,~data.columns.duplicated()].copy()
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,customerEmail,455,0.730337,0,0.000000,161,object
1,customerPhone,455,0.730337,0,0.000000,168,object
2,customerDevice,455,0.730337,0,0.000000,168,object
3,customerIPAddress,455,0.730337,0,0.000000,165,object
4,customerBillingAddress,455,0.730337,0,0.000000,166,object
5,orderId,145,0.232745,0,0.000000,478,object
6,orderAmount,145,0.232745,0,0.000000,67,float64
7,orderState,145,0.232745,0,0.000000,3,object
8,orderShippingAddress,145,0.232745,0,0.000000,274,object
9,paymentMethodId,281,0.451043,0,0.000000,342,object


## 10. Extraer proveedor y dominio de email

In [12]:
def get_provider(email):
    try:
        aux = email.split('@')[1]
        provider = aux.split('.')[0]
    except:
        return 'weird'

    if provider in ['gmail', 'yahoo', 'hotmail']:
        return provider
    return 'other'


def get_domain(email):
    try:
        aux = email.split('@')[1]
        domain = aux.split('.')[1]
    except:
        return 'weird'

    return domain


data['emailProvider'] = data['customerEmail'].apply(get_provider)
data['emailDomain'] = data['customerEmail'].apply(get_domain)
data.head()

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,orderId,orderAmount,orderState,orderShippingAddress,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionId,transactionAmount,transactionFailed,fraudulent,emailProvider,emailDomain
0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",vjbdvd,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",wt07xm68b,True,card,JCB 16 digit,Citizens First Banks,a9lcj51r,18,False,False,yahoo,com
1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,yp6x27,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks,y4wcv03i,26,False,True,yahoo,com
2,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",nlghpa,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",6krszxc05,False,card,VISA 16 digit,Vertex Bancorp,5mi94sfw,45,False,False,yahoo,com
3,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",uw0eeb,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,br8ba1nu,23,False,False,yahoo,com
4,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",bn44oh,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",m52tx8e1s,False,card,Mastercard,Vertex Bancorp,a33145ss,43,True,True,other,com


## 11. Eliminamos la columna customerEmail

In [13]:
data = data.drop(['customerEmail'], axis=1)
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,customerPhone,455,0.730337,0,0.000000,168,object
1,customerDevice,455,0.730337,0,0.000000,168,object
2,customerIPAddress,455,0.730337,0,0.000000,165,object
3,customerBillingAddress,455,0.730337,0,0.000000,166,object
4,orderId,145,0.232745,0,0.000000,478,object
5,orderAmount,145,0.232745,0,0.000000,67,float64
6,orderState,145,0.232745,0,0.000000,3,object
7,orderShippingAddress,145,0.232745,0,0.000000,274,object
8,paymentMethodId,281,0.451043,0,0.000000,342,object
9,paymentMethodRegistrationFailure,281,0.451043,302,0.484751,2,object


## 12. IP Address

In [14]:
## Create a function with column customerIPAddress clasify as IPV4 or IPV6, and return the clasification in a new column
import ipaddress

def get_ip_type(ip):
    try:
        ipaddress.ip_address(ip)
        return ipaddress.ip_address(ip).version
    except:
        return None

data['customerIPAddress'] = data['customerIPAddress'].astype('str')
data['customerIPAddressVersion'] = data['customerIPAddress'].apply(get_ip_type)
data.head()

,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,orderId,orderAmount,orderState,orderShippingAddress,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionId,transactionAmount,transactionFailed,fraudulent,emailProvider,emailDomain,customerIPAddressVersion
0,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",vjbdvd,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",wt07xm68b,True,card,JCB 16 digit,Citizens First Banks,a9lcj51r,18,False,False,yahoo,com,4.0
1,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,yp6x27,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks,y4wcv03i,26,False,True,yahoo,com,4.0
2,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",nlghpa,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",6krszxc05,False,card,VISA 16 digit,Vertex Bancorp,5mi94sfw,45,False,False,yahoo,com,6.0
3,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",uw0eeb,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,br8ba1nu,23,False,False,yahoo,com,6.0
4,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",bn44oh,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",m52tx8e1s,False,card,Mastercard,Vertex Bancorp,a33145ss,43,True,True,other,com,4.0


## 13. Eliminamos la columna customerIPAddress

In [15]:
data = data.drop(['customerIPAddress'], axis=1)
status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,customerPhone,455,0.730337,0,0.000000,168,object
1,customerDevice,455,0.730337,0,0.000000,168,object
2,customerBillingAddress,455,0.730337,0,0.000000,166,object
3,orderId,145,0.232745,0,0.000000,478,object
4,orderAmount,145,0.232745,0,0.000000,67,float64
5,orderState,145,0.232745,0,0.000000,3,object
6,orderShippingAddress,145,0.232745,0,0.000000,274,object
7,paymentMethodId,281,0.451043,0,0.000000,342,object
8,paymentMethodRegistrationFailure,281,0.451043,302,0.484751,2,object
9,paymentMethodType,281,0.451043,0,0.000000,4,object


In [16]:
data.shape

(623, 19)

## 14.customerBillingAddress vrs. orderShippingAddress

In [18]:
def extract_city_name(df_column):
    city_list = []

    for address in df_column:
        try:
            """
            Si address = "5493 Jones Islands\nBrownside, CA 51896"

            aux = "Brownside, CA 51896"
            city_with_number = "CA 51896"
            city = "CA"
            """
            aux = address.split('\n')[1]
            city_with_number = aux.split(', ')[1]
            city = city_with_number.split(' ')[0]

            city_list.append(city)

        except:
            # Sino aparece el nombre de la ciudad asignamos: unknown
            city_list.append("unknown")

    return city_list

In [19]:
data["customerBillingAddress"] = extract_city_name(data["customerBillingAddress"])
data["orderShippingAddress"] = extract_city_name(data["orderShippingAddress"])

In [20]:
data.head()

,customerPhone,customerDevice,customerBillingAddress,orderId,orderAmount,orderState,orderShippingAddress,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionId,transactionAmount,transactionFailed,fraudulent,emailProvider,emailDomain,customerIPAddressVersion
0,400-108-5415,yyeiaxpltf82440jnb3v,CA,vjbdvd,18.0,pending,CA,wt07xm68b,True,card,JCB 16 digit,Citizens First Banks,a9lcj51r,18,False,False,yahoo,com,4.0
1,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,NM,yp6x27,26.0,fulfilled,CA,y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks,y4wcv03i,26,False,True,yahoo,com,4.0
2,024.420.0375,4m7h5ipl1shyavt6vv2r,PR,nlghpa,45.0,fulfilled,CA,6krszxc05,False,card,VISA 16 digit,Vertex Bancorp,5mi94sfw,45,False,False,yahoo,com,6.0
3,670.664.8168x94985,slovx60t0i558may4ks0,FM,uw0eeb,23.0,fulfilled,NM,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,br8ba1nu,23,False,False,yahoo,com,6.0
4,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,TN,bn44oh,43.0,fulfilled,FM,m52tx8e1s,False,card,Mastercard,Vertex Bancorp,a33145ss,43,True,True,other,com,4.0


In [21]:
same_city = []

for index in range(len(data)):
    billing_address = data["customerBillingAddress"][index]
    order_address = data["orderShippingAddress"][index]

    response = "unknown"
    if billing_address != "unknown" and order_address != "unknown":
        if billing_address == order_address:
            response = "yes"
        else:
            response = "no"

    same_city.append(response)

In [22]:
data["sameCity"] = same_city

In [23]:
data.head()

,customerPhone,customerDevice,customerBillingAddress,orderId,orderAmount,orderState,orderShippingAddress,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionId,transactionAmount,transactionFailed,fraudulent,emailProvider,emailDomain,customerIPAddressVersion,sameCity
0,400-108-5415,yyeiaxpltf82440jnb3v,CA,vjbdvd,18.0,pending,CA,wt07xm68b,True,card,JCB 16 digit,Citizens First Banks,a9lcj51r,18,False,False,yahoo,com,4.0,yes
1,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,NM,yp6x27,26.0,fulfilled,CA,y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks,y4wcv03i,26,False,True,yahoo,com,4.0,no
2,024.420.0375,4m7h5ipl1shyavt6vv2r,PR,nlghpa,45.0,fulfilled,CA,6krszxc05,False,card,VISA 16 digit,Vertex Bancorp,5mi94sfw,45,False,False,yahoo,com,6.0,no
3,670.664.8168x94985,slovx60t0i558may4ks0,FM,uw0eeb,23.0,fulfilled,NM,5z1szj2he,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,br8ba1nu,23,False,False,yahoo,com,6.0,no
4,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,TN,bn44oh,43.0,fulfilled,FM,m52tx8e1s,False,card,Mastercard,Vertex Bancorp,a33145ss,43,True,True,other,com,4.0,no


In [24]:
data = data.drop(["customerBillingAddress", "orderShippingAddress"], axis = 1)

In [25]:
data = data.drop(['customerPhone','customerDevice', 'orderId', 'transactionId', 'paymentMethodId'], axis=1)

In [26]:
data.head()

,orderAmount,orderState,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,transactionAmount,transactionFailed,fraudulent,emailProvider,emailDomain,customerIPAddressVersion,sameCity
0,18.0,pending,True,card,JCB 16 digit,Citizens First Banks,18,False,False,yahoo,com,4.0,yes
1,26.0,fulfilled,True,bitcoin,VISA 16 digit,Solace Banks,26,False,True,yahoo,com,4.0,no
2,45.0,fulfilled,False,card,VISA 16 digit,Vertex Bancorp,45,False,False,yahoo,com,6.0,no
3,23.0,fulfilled,False,card,Diners Club / Carte Blanche,His Majesty Bank Corp.,23,False,False,yahoo,com,6.0,no
4,43.0,fulfilled,False,card,Mastercard,Vertex Bancorp,43,True,True,other,com,4.0,no


In [27]:
filename = "../data/customer_dataset.csv"
data.to_csv(filename, index = False)